In [1]:
# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import re
import time
import mmh3
import math
import datasketch

In [3]:
# Підготовка даних

k = 1 # коефіцієнт  масштабування списку адрес для тестування на великих масивах

ip_pattern = re.compile(r'\b(?:25[0-5]|2[0-4]\d|1\d{2}|[1-9]?\d)\.' 
                     r'(?:25[0-5]|2[0-4]\d|1\d{2}|[1-9]?\d)\.' 
                     r'(?:25[0-5]|2[0-4]\d|1\d{2}|[1-9]?\d)\.' 
                     r'(?:25[0-5]|2[0-4]\d|1\d{2}|[1-9]?\d)\b')

with open('work/lms-stage-access.log', 'r', encoding='utf-8') as f:
    text = f.readlines()

print('Кількість рядків:', len(text))
ip_list = []

# ip_set = set()
for line in text:
    ip_list.append(ip_pattern.findall(line)[0]) # вибираємо першу знайдену ip-адресу в кожному рядку (remote address)
    
ip_list = ip_list * k
    
print('Загальна кількість витягнутих з текста неунікальних ip-адрес:', len(ip_list))

Кількість рядків: 29553
Загальна кількість витягнутих з текста неунікальних ip-адрес: 29553


In [4]:
# Тестуємо швидкість виконання коду на множинах set (структура даних в Python)

start_time = time.monotonic()    
ip_set = set(ip_list)
set_cardinality = len(ip_set)
set_duration = time.monotonic()  - start_time


# print(f'Час виконання: {set_duration:.6f} секунд')
# print(f'Кількість унікальних IP-адрес: {set_cardinality}')

In [5]:
# Тестуємо швидкість виконання коду на кастомній структурі даних, яка працює як множина set, але як скрипт Python, а не як вбудована структура даних на C

custom_set_as_list = []

start_time = time.monotonic()    
for ip in ip_list:
	if ip not in custom_set_as_list:
		custom_set_as_list.append(ip)
custom_set_cardinality = len(custom_set_as_list)
custom_set_duration = time.monotonic()  - start_time


# print(f'Час виконання: {custom_set_duration:.6f} секунд')
# print(f'Кількість унікальних IP-адрес: {custom_set_cardinality}')

In [6]:
# Тестуємо швидкість виконання коду за допомогою алгоритму HyperLogLog (реалізація на Python без використання сторонніх бібліотек), код з конспекту лекції

class PythonHyperLogLog:
    def __init__(self, p=5):
        self.p = p
        self.m = 1 << p
        self.registers = [0] * self.m
        self.alpha = self._get_alpha()
        self.small_range_correction = 5 * self.m / 2  # Поріг для малих значень

    def _get_alpha(self):
        if self.p <= 16:
            return 0.673
        elif self.p == 32:
            return 0.697
        else:
            return 0.7213 / (1 + 1.079 / self.m)

    def add(self, item):
        x = mmh3.hash(str(item), signed=False)
        j = x & (self.m - 1)
        w = x >> self.p
        self.registers[j] = max(self.registers[j], self._rho(w))

    def _rho(self, w):
        return len(bin(w)) - 2 if w > 0 else 32

    def count(self):
        Z = sum(2.0 ** -r for r in self.registers)
        E = self.alpha * self.m * self.m / Z
        
        if E <= self.small_range_correction:
            V = self.registers.count(0)
            if V > 0:
                return self.m * math.log(self.m / V)
        
        return E

# Приклад використання
phll = PythonHyperLogLog(p=14)

start_time = time.monotonic()  
for ip in ip_list:
    phll.add(ip)
phhl_estimated_cardinality = phll.count()
phhl_duration = time.monotonic() - start_time

# print(f'Час виконання: {phhl_duration:.6f} секунд')
# print(f"Оцінена кардинальність: {phhl_estimated_cardinality}")

In [7]:
# Тестуємо швидкість виконання коду за допомогою алгоритму HyperLogLog (реалізація за допомогою бібліотеки datasketch, код з конспекту лекції

# Ініціалізація HyperLogLog
ds_hll = datasketch.HyperLogLog(p=14)


start_time = time.monotonic() 
for data in ip_list:
    ds_hll.update(data.encode('utf-8'))
ds_hll_estimated_cardinality = ds_hll.count()
ds_hhl_duration = time.monotonic()  - start_time

# print(f'Час виконання: {ds_hll_estimated_cardinality:.6f} секунд')
# print(f"Оцінена кардинальність: {ds_hhl_duration}")

In [8]:
print('Результати порівняння:')
print(f'Python set              : {set_cardinality:>18} унікальних IP-адрес, час виконання: {set_duration:.6f} секунд')
print(f'Кастомний set           : {custom_set_cardinality:>18} унікальних IP-адрес, час виконання: {custom_set_duration:.6f} секунд')
print(f'HyperLogLog (Python)    : {phhl_estimated_cardinality:>18} унікальних IP-адрес, час виконання: {phhl_duration:.6f} секунд')
print(f'HyperLogLog (datasketch): {ds_hll_estimated_cardinality:>18} унікальних IP-адрес, час виконання: {ds_hhl_duration:.6f} секунд')

Результати порівняння:
Python set              :                 28 унікальних IP-адрес, час виконання: 0.001318 секунд
Кастомний set           :                 28 унікальних IP-адрес, час виконання: 0.008700 секунд
HyperLogLog (Python)    : 28.023953075428718 унікальних IP-адрес, час виконання: 0.042895 секунд
HyperLogLog (datasketch): 28.023953075428718 унікальних IP-адрес, час виконання: 0.093938 секунд


Висновки:
=========
1. Різниця між __Python set__ та __Кастомний set__ - очикувана.
2. Різниця між __Кастомний set__ та __HyperLogLog (Python)__ / __HyperLogLog (datasketch)__ - зворотня очикуваному. Тестування на масивах даних різного масштабу (наприклад, k = 1000) не показали суттєвої різниці. Тобто справа не в розміру масиву даних.
3. Скоріш за все причина такого результату, що нативні функції Python реалізовіані на C (інтерпретатор Python написаний на C), на відміну від обох реалізацій HyperLogLog (datasketch теж написаний на Python), хоча і з використанням Numpy.